# Linked Geodata poster
- Author: David Swinkels
- Date 24 August 2018


# Introduction

<div style="text-align: justify"> Linked data is a set of design principles to publish data on the web and to perform data management to save relations of features in a graph database. Linked data provides the best current method to make links in data understandable by machines and humans via the web and locally. Linked geodata adds a spatial component to linked data. </div>

This notebook will showcase some theory and how to make linked geodata.

In [123]:
# Import all libraries
import rdflib
import geopandas as gpd
import pandas as pd
import requests
import folium
from folium.map import *
import shapely.wkt
from IPython.display import Latex, display

from rdflib import Graph, Namespace, RDF, RDFS, OWL, Literal, URIRef

# Concept
<div style="text-align: justify"> The concept of linked data is elegantly simple; storing relations and sharing individual resources via unique IRIs. Linked data is often visualized in a graph to show relations (see figure below) and the data is stored in relations as triples instead of values in columns and rows. A triple consists of three resources: a subject, a predicate and an object. The predicate defines the type of relation the subject and object have. Furthermore, each resource, whether that is a subject, predicate or object, is stored as an IRI. An IRI is an International Resource Identifier and similar as a Uniform Resource Identifier [URI] or Uniform Resource Locator [URL], but an IRI also allows international characters. Objects can optionally be stored as literals, such as a string, integer or date. Triples store relations in a framework called Resource Description Framework [RDF]. RDF supports data merging with different schemas and evolving schemas. Linked data allows a query to find out who the grandfather of Angie is based on semantic relations. </div>

<img src="./images/LinkedDataGraph.png">

<div style="text-align: justify"> An RDF stores a vocabulary and instances. A vocabulary defines the concepts and relationships in an area of concern. Vocabularies store classes and properties.Instances are entities of classes. </div>

|Triple notation: | subject | predicate | object |
|------|------|------|------|
|Class definition: | <dbo:Person> | <rdf:type> | <rdfs:Class> |
|Property definition: | <dbo:spouse> | <rdf:type> | <owl:ObjectProperty>|
|Entity instance: | <person:Maggie> | <rdf:type> | <dbo:Person>|
|Relation instance: | <person:Maggie> | <dbo:spouse> | <person:Peter>|
|Value instance: | <person:Angie> | <dbo:birthDate> | 15 May 2001|




<div style="text-align: justify"> Long IRIs, that link to unique resources, are shortened with a prefix to improve readability and save data storage. IRI 'http://www.dbpedia.org/ontology/Person' is shortened to dbo:Person. Dbo is the shortened prefix that links to the namespace 'http://www.dbpedia.org/ontology/' and Person is the unique ID or Class. Instead of having to say 'http://www.dbpedia.org/ontology/Mother', you can refer to the IRI as dbo:Mother. The prefix and namespaces are defined in the RDF file before the triple instances. The graph is stored as triples with namespaces and prefixes. These namespaces and prefixes can be re-used. This is very powerful, because the semantics are persistent and are used across domains. One public ontology is the building ontology from the land administration office in the Netherlands. This ontology stores all classes, properties, concepts, regulations and restrictions of building data as can be seen below. </div>


In [124]:
# Download building Ontology
tripleHeader = {'Accept': 'text/turtle'}
response = requests.get("  http://bag.basisregistraties.overheid.nl/def/bag", headers=tripleHeader)
#print(response.text)

The downloaded building ontology is very complete. We will get some more hands-on experience with linked data by making our own ontology. The goal is to define classes and relations to be able to integrate multiple types of data. The building data of the BAG is seperated in buildings, addresses and building functions. An ontology makes it possible to create entities that are connected.

In [125]:
# Create empty graph
g = Graph()

Now create some Namespace for the objects and the concepts. The namespaces of the objects will be used later to store buildings, addresses and building functions. The concepts are binded to the graph as a prefix.

In [126]:
buildingNamespace = Namespace('https://bag.nl/id/building/')
addressNamespace = Namespace('https://bag.nl/id/address/')
buildingFunctionNamespace = Namespace('https://bag.nl/id/buildingfunction/')
geosparql = Namespace('http://www.opengis.net/ont/geosparql#')
g.bind(prefix="geosparql", namespace=geosparql)
bag = Namespace('http://bag.nl/def/bag#')
g.bind(prefix="bag", namespace=bag)


Let's check what is in the graph.

In [127]:
print(g.serialize(format="turtle").decode("utf-8"))


@prefix bag: <http://bag.nl/def/bag#> .
@prefix geosparql: <http://www.opengis.net/ont/geosparql#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .




The added bag prefix is visible. Furthermore, the graph shows the standard prefixes RDF, RDFS, XML and XSD. To give a very brief explanation. RDF is used to define classes, subject, predicates and objects, RDFS to define subclasses, properties, domains, ranges and labels, XML is a standard to write markup language with prefixes and XSD defines how datatypes are saved. More information can be found on W3C specifications. 

Now some classes can be added. The OWL ontology is used to define the classes together with RDF. Let's make the classes for building, address and building function. Later instances can be added to these classes to become instance of that specific class.

In [128]:
# Create building, address and building function class
g.add((bag.Building, RDF.type, OWL.Class))
g.add((bag.Address, RDF.type, OWL.Class))
g.add((bag.BuildingFunction, RDF.type, OWL.Class))

After adding the classes, the properties can be added. There are two types of properties: object and datatype. Object properties links two instances of different classes and will be used as a predicate later. For example an object property is the predicate "main address" that links the subject building and an object address. The domain and range can be specified of an object property to specify which subjects and objects can. The domain specifies and limits the possible subjects belonging to the predicate. The range does the same for objects. So the predicate "hasFunction", that is an object property, always has the subject "Building" and object "BuildingFunction".

In [129]:
# Create ObjectProperties that link buildings to addresses and functions
g.add((bag.hasFunction, RDF.type, OWL.ObjectProperty))
g.add((bag.hasFunction, RDFS.domain, bag.Building))
g.add((bag.hasFunction, RDFS.range, bag.BuildingFunction))
g.add((bag.hasAddress, RDF.type, OWL.ObjectProperty))
g.add((bag.hasAddress, RDFS.domain, bag.Building))
g.add((bag.hasAddress, RDFS.range, bag.Address))

Datatype properties are the predicates that link subjects to a specific value, string or date. Example datatype properties are status, function or address.


In [130]:
g.add((bag.hasStatus, RDF.type, OWL.DatatypeProperty))
g.add((bag.hasStatus, RDFS.label, Literal("Status of a building", lang="En")))
g.add((bag.hasStatus, RDFS.domain, bag.Building))
g.add((bag.function, RDF.type, OWL.DatatypeProperty))
g.add((bag.function, RDFS.label, Literal("Function of a building function. Each building can have multiple functions.", lang="En")))
g.add((bag.function, RDFS.domain, bag.BuildingFunction))
g.add((bag.address, RDF.type, OWL.DatatypeProperty))
g.add((bag.address, RDFS.label, Literal("Postal address of building", lang="En")))
g.add((bag.address, RDFS.domain, bag.Address))

Okay we did add more triples to the graph. Check what is in there.

In [131]:
print(g.serialize(format='turtle').decode("utf-8"))

@prefix bag: <http://bag.nl/def/bag#> .
@prefix geosparql: <http://www.opengis.net/ont/geosparql#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

bag:address a <http://www.w3.org/2002/07/owl#DatatypeProperty> ;
    rdfs:label "Postal address of building"@En ;
    rdfs:domain bag:Address .

bag:function a <http://www.w3.org/2002/07/owl#DatatypeProperty> ;
    rdfs:label "Function of a building function. Each building can have multiple functions."@En ;
    rdfs:domain bag:BuildingFunction .

bag:hasAddress a <http://www.w3.org/2002/07/owl#ObjectProperty> ;
    rdfs:domain bag:Building ;
    rdfs:range bag:Address .

bag:hasFunction a <http://www.w3.org/2002/07/owl#ObjectProperty> ;
    rdfs:domain bag:Building ;
    rdfs:range bag:BuildingFunction .

bag:hasStatus a <http://www.w3.org/2002/07/owl#DatatypeProperty

We can see that the Properties and Classes have been added to the graph. Let's add some data.


In [132]:
buildings = gpd.read_file("./data/building.geojson")
addresses = gpd.read_file("./data/address.geojson")
buildingFunctions = gpd.read_file("./data/buildingfunction.geojson")
rdnew_crs = {'init': 'epsg:28992'}
buildings.crs = rdnew_crs

In [133]:
wgs84_crs = {'init': 'epsg:4326'}
buildings.to_crs(wgs84_crs, inplace=True)
addresses.to_crs(wgs84_crs, inplace=True)
buildingFunctions.to_crs(wgs84_crs, inplace=True)

In [134]:
m = folium.Map([52.0928, 5.1062], zoom_start=18, tiles='cartodbpositron')

folium.GeoJson(buildings, name="Buildings", style_function=lambda feature: {'fillColor': "Red", 'color': "Black"}).add_to(m)
folium.GeoJson(addresses, name="Addresses", style_function=lambda feature: {'fill_color': "Blue"}).add_to(m)
folium.GeoJson(buildingFunctions, name="BuildingFunctions", style_function=lambda feature: {'fill_color': "Green"}).add_to(m)
folium.LatLngPopup().add_to(m)
folium.LayerControl().add_to(m)
m

<div style="width:100%;"><div style="position:relative;width:100%;height:0;padding-bottom:60%;"><iframe src="data:text/html;charset=utf-8;base64,PCFET0NUWVBFIGh0bWw+CjxoZWFkPiAgICAKICAgIDxtZXRhIGh0dHAtZXF1aXY9ImNvbnRlbnQtdHlwZSIgY29udGVudD0idGV4dC9odG1sOyBjaGFyc2V0PVVURi04IiAvPgogICAgPHNjcmlwdD5MX1BSRUZFUl9DQU5WQVM9ZmFsc2U7IExfTk9fVE9VQ0g9ZmFsc2U7IExfRElTQUJMRV8zRD1mYWxzZTs8L3NjcmlwdD4KICAgIDxzY3JpcHQgc3JjPSJodHRwczovL2Nkbi5qc2RlbGl2ci5uZXQvbnBtL2xlYWZsZXRAMS4yLjAvZGlzdC9sZWFmbGV0LmpzIj48L3NjcmlwdD4KICAgIDxzY3JpcHQgc3JjPSJodHRwczovL2FqYXguZ29vZ2xlYXBpcy5jb20vYWpheC9saWJzL2pxdWVyeS8xLjExLjEvanF1ZXJ5Lm1pbi5qcyI+PC9zY3JpcHQ+CiAgICA8c2NyaXB0IHNyYz0iaHR0cHM6Ly9tYXhjZG4uYm9vdHN0cmFwY2RuLmNvbS9ib290c3RyYXAvMy4yLjAvanMvYm9vdHN0cmFwLm1pbi5qcyI+PC9zY3JpcHQ+CiAgICA8c2NyaXB0IHNyYz0iaHR0cHM6Ly9jZG5qcy5jbG91ZGZsYXJlLmNvbS9hamF4L2xpYnMvTGVhZmxldC5hd2Vzb21lLW1hcmtlcnMvMi4wLjIvbGVhZmxldC5hd2Vzb21lLW1hcmtlcnMuanMiPjwvc2NyaXB0PgogICAgPGxpbmsgcmVsPSJzdHlsZXNoZWV0IiBocmVmPSJodHRwczovL2Nkbi5qc2RlbGl2ci5uZXQvbnBtL2xlYWZsZXRAMS4yLjAvZGlzdC9sZWFmbGV0LmNzcyIvPgogICAgPGxpbmsgcmVsPSJzdHlsZXNoZWV0IiBocmVmPSJodHRwczovL21heGNkbi5ib290c3RyYXBjZG4uY29tL2Jvb3RzdHJhcC8zLjIuMC9jc3MvYm9vdHN0cmFwLm1pbi5jc3MiLz4KICAgIDxsaW5rIHJlbD0ic3R5bGVzaGVldCIgaHJlZj0iaHR0cHM6Ly9tYXhjZG4uYm9vdHN0cmFwY2RuLmNvbS9ib290c3RyYXAvMy4yLjAvY3NzL2Jvb3RzdHJhcC10aGVtZS5taW4uY3NzIi8+CiAgICA8bGluayByZWw9InN0eWxlc2hlZXQiIGhyZWY9Imh0dHBzOi8vbWF4Y2RuLmJvb3RzdHJhcGNkbi5jb20vZm9udC1hd2Vzb21lLzQuNi4zL2Nzcy9mb250LWF3ZXNvbWUubWluLmNzcyIvPgogICAgPGxpbmsgcmVsPSJzdHlsZXNoZWV0IiBocmVmPSJodHRwczovL2NkbmpzLmNsb3VkZmxhcmUuY29tL2FqYXgvbGlicy9MZWFmbGV0LmF3ZXNvbWUtbWFya2Vycy8yLjAuMi9sZWFmbGV0LmF3ZXNvbWUtbWFya2Vycy5jc3MiLz4KICAgIDxsaW5rIHJlbD0ic3R5bGVzaGVldCIgaHJlZj0iaHR0cHM6Ly9yYXdnaXQuY29tL3B5dGhvbi12aXN1YWxpemF0aW9uL2ZvbGl1bS9tYXN0ZXIvZm9saXVtL3RlbXBsYXRlcy9sZWFmbGV0LmF3ZXNvbWUucm90YXRlLmNzcyIvPgogICAgPHN0eWxlPmh0bWwsIGJvZHkge3dpZHRoOiAxMDAlO2hlaWdodDogMTAwJTttYXJnaW46IDA7cGFkZGluZzogMDt9PC9zdHlsZT4KICAgIDxzdHlsZT4jbWFwIHtwb3NpdGlvbjphYnNvbHV0ZTt0b3A6MDtib3R0b206MDtyaWdodDowO2xlZnQ6MDt9PC9zdHlsZT4KICAgIAogICAgPHN0eWxlPiNtYXBfNWFhYTVlNzZhNzc5NGM3N2IyMGI4NzlmNTU0YjBjYTQgewogICAgICAgIHBvc2l0aW9uOiByZWxhdGl2ZTsKICAgICAgICB3aWR0aDogMTAwLjAlOwogICAgICAgIGhlaWdodDogMTAwLjAlOwogICAgICAgIGxlZnQ6IDAuMCU7CiAgICAgICAgdG9wOiAwLjAlOwogICAgICAgIH0KICAgIDwvc3R5bGU+CjwvaGVhZD4KPGJvZHk+ICAgIAogICAgCiAgICA8ZGl2IGNsYXNzPSJmb2xpdW0tbWFwIiBpZD0ibWFwXzVhYWE1ZTc2YTc3OTRjNzdiMjBiODc5ZjU1NGIwY2E0IiA+PC9kaXY+CjwvYm9keT4KPHNjcmlwdD4gICAgCiAgICAKICAgIAogICAgICAgIHZhciBib3VuZHMgPSBudWxsOwogICAgCgogICAgdmFyIG1hcF81YWFhNWU3NmE3Nzk0Yzc3YjIwYjg3OWY1NTRiMGNhNCA9IEwubWFwKAogICAgICAgICdtYXBfNWFhYTVlNzZhNzc5NGM3N2IyMGI4NzlmNTU0YjBjYTQnLCB7CiAgICAgICAgY2VudGVyOiBbNTIuMDkyOCwgNS4xMDYyXSwKICAgICAgICB6b29tOiAxOCwKICAgICAgICBtYXhCb3VuZHM6IGJvdW5kcywKICAgICAgICBsYXllcnM6IFtdLAogICAgICAgIHdvcmxkQ29weUp1bXA6IGZhbHNlLAogICAgICAgIGNyczogTC5DUlMuRVBTRzM4NTcsCiAgICAgICAgem9vbUNvbnRyb2w6IHRydWUsCiAgICAgICAgfSk7CgogICAgCiAgICAKICAgIHZhciB0aWxlX2xheWVyXzI1Yzg5NWU2ZjVlZjQxZTFhNWVkOGIxNzVmY2E0OWM3ID0gTC50aWxlTGF5ZXIoCiAgICAgICAgJ2h0dHBzOi8vY2FydG9kYi1iYXNlbWFwcy17c30uZ2xvYmFsLnNzbC5mYXN0bHkubmV0L2xpZ2h0X2FsbC97en0ve3h9L3t5fS5wbmcnLAogICAgICAgIHsKICAgICAgICAiYXR0cmlidXRpb24iOiBudWxsLAogICAgICAgICJkZXRlY3RSZXRpbmEiOiBmYWxzZSwKICAgICAgICAibWF4TmF0aXZlWm9vbSI6IDE4LAogICAgICAgICJtYXhab29tIjogMTgsCiAgICAgICAgIm1pblpvb20iOiAwLAogICAgICAgICJub1dyYXAiOiBmYWxzZSwKICAgICAgICAic3ViZG9tYWlucyI6ICJhYmMiCn0pLmFkZFRvKG1hcF81YWFhNWU3NmE3Nzk0Yzc3YjIwYjg3OWY1NTRiMGNhNCk7CiAgICAKICAgICAgICAKICAgICAgICB2YXIgZ2VvX2pzb25fOTYzZTZlM2NkNzgxNGM5MGExMTljODkyYzM2NTA2YmMgPSBMLmdlb0pzb24oCiAgICAgICAgICAgIHsiYmJveCI6IFs1LjEwNTYwODUxNDIzMzQ5NywgNTIuMDkyMTAwMTAzNjU4ODgsIDUuMTA2ODYyNzg0MDY4NjExLCA1Mi4wOTMwMzUxNjQxNjcyNzZdLCAiZmVhdHVyZXMiOiBbeyJiYm94IjogWzUuMTA2NTI5NTczODkzODMzLCA1Mi4wOTIzNzAyNjY0MjEzNCwgNS4xMDY3NDYyMTQ2NDk4MzEsIDUyLjA5MjQ0MjIyODU4MTRdLCAiZ2VvbWV0cnkiOiB7ImNvb3JkaW5hdGVzIjogW1tbNS4xMDY1NDk4Nzk4MjI3ODQsIDUyLjA5MjM3MDI2NjQyMTM0XSwgWzUuMTA2NTI5NTczODkzODMzLCA1Mi4wOTI0MTM0NTA2OTI2OTZ

Okay we created a map with buildings, addresses and building functions. How can these be linked?
The related unique IDs are the key to link the datasets. So in the GeoDataframe we should check which unique IDs match:

In [135]:
print("--------------------------------------- Building -------------------------------------")
print(buildings.iloc[0])
print("--------------------------------------- Address -------------------------------------")
print(addresses.iloc[0])
print("--------------------------------------- Building function -------------------------------------")
print(buildingFunctions.iloc[0])

--------------------------------------- Building -------------------------------------
FID                                                       86881
pandid                                         0344100000005566
bouwjaar                                                   1915
officieel                                                     N
status                                          Pand in gebruik
begindatum                                     2010061000000000
einddatum                                                      
documentda                                             20100610
documentnu                                       SO.10.04.70.82
aanduiding                                                    N
shape_Leng                                              38.3041
shape_Area                                              62.8222
verblijfso                 03440100000599312010061000000000N000
adresseerb                                     0344010000059931
geometry      POL

Some of the explanations are in Dutch. So let me help. The buildings and addresses can be linked by the "addreseerb" column of the building and the "identifica" of the address. The building and building function can be linked on the "verblijfso" of the building and "versieiden" of the building function. We iterate over the buildings and give all the links as object properties and datatype properties. Same for the addresses and building function.


In [136]:
for index, building in buildings.iterrows():
    buildingIRI = buildingNamespace + building['pandid']
    g.add((URIRef(buildingIRI), RDF.type, bag.Building))
    g.add((URIRef(buildingIRI), bag.hasStatus, Literal(building['status'])))
    if building['adresseerb'] is not " ":
        addressIRI = addressNamespace + building['adresseerb']
        g.add((URIRef(buildingIRI), bag.hasAddress, URIRef(addressIRI)))
    if building['verblijfso'] is not " ":
        buildingFunctionIRI = buildingFunctionNamespace + building['verblijfso']
        g.add((URIRef(buildingIRI), bag.hasFunction, URIRef(buildingFunctionIRI)))
    g.add((URIRef(buildingIRI), geosparql.asWKT, Literal(building['geometry'])))

for index, address in addresses.iterrows():
    addressIRI = addressNamespace + address['adressee_2']
    g.add((URIRef(addressIRI), RDF.type, bag.Address))
    addressText = "%s %s %s %s" % (address['openbareru'], address['huisnummer'], 
                                   address['postcode'], address['woonplaats'])
    g.add((URIRef(addressIRI), bag.address, Literal(addressText)))
    g.add((URIRef(addressIRI), geosparql.asWKT, Literal(address['geometry'])))
    
for index, buildingFunction in buildingFunctions.iterrows():
    buildingFunctionIRI = buildingFunctionNamespace + buildingFunction['versieiden']
    g.add((URIRef(buildingFunctionIRI), RDF.type, bag.BuildingFunction))
    g.add((URIRef(buildingFunctionIRI), bag.function, Literal(str(buildingFunction['gebruiksdo']))))   
    if buildingFunction['gebruiks_1'] is not " ":
        g.add((URIRef(buildingFunctionIRI), bag.function, Literal(str(buildingFunction['gebruiks_1']))))
    g.add((URIRef(buildingFunctionIRI), geosparql.asWKT, Literal(buildingFunction['geometry'])))

The buildings, addresses and building functions are added to the same schema! This means two things. Firstly, the schema of the concepts and the data of physical objects are stored in the same data! Secondly, data of various physical objects is stored in the same schema! Just think how mind blowing that is. Everything. I mean everyyyyyything can be stored in the graph and connected to related objects. Let's have a look at the related objects in the graph.

In [137]:
print(g.serialize(format='turtle').decode("utf-8"))

@prefix bag: <http://bag.nl/def/bag#> .
@prefix geosparql: <http://www.opengis.net/ont/geosparql#> .
@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .
@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .
@prefix xml: <http://www.w3.org/XML/1998/namespace> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

bag:address a <http://www.w3.org/2002/07/owl#DatatypeProperty> ;
    rdfs:label "Postal address of building"@En ;
    rdfs:domain bag:Address .

bag:function a <http://www.w3.org/2002/07/owl#DatatypeProperty> ;
    rdfs:label "Function of a building function. Each building can have multiple functions."@En ;
    rdfs:domain bag:BuildingFunction .

bag:hasAddress a <http://www.w3.org/2002/07/owl#ObjectProperty> ;
    rdfs:domain bag:Building ;
    rdfs:range bag:Address .

bag:hasFunction a <http://www.w3.org/2002/07/owl#ObjectProperty> ;
    rdfs:domain bag:Building ;
    rdfs:range bag:BuildingFunction .

bag:hasStatus a <http://www.w3.org/2002/07/owl#DatatypeProperty

By scrolling through the buildings, the connections to addresses and building functions can be seen. These connections can also be queried with SPARQL.

In [138]:
response = g.query("SELECT * WHERE { ?s ?p ?o .} LIMIT 10")

for row in response:
    print("%s %s %s" % row)


Westplein 107 3531BL Utrecht http://bag.nl/def/bag#address https://bag.nl/id/address/0344010000031478
http://bag.nl/def/bag#Address http://www.w3.org/1999/02/22-rdf-syntax-ns#type https://bag.nl/id/address/0344010000044632
POINT (5.106405603262396 52.09225702380836) http://www.opengis.net/ont/geosparql#asWKT https://bag.nl/id/address/0344010000044948
POLYGON ((5.106443874587443 52.09227156775525, 5.106465095890107 52.09222668693219, 5.106318713236053 52.09220263678149, 5.10630949175248 52.09222344016747, 5.106224466063757 52.09220912631888, 5.106214193834804 52.09223226409122, 5.106299190436911 52.09224656888979, 5.106443874587443 52.09227156775525)) http://www.opengis.net/ont/geosparql#asWKT https://bag.nl/id/building/0344100000063761
http://bag.nl/def/bag#BuildingFunction http://www.w3.org/1999/02/22-rdf-syntax-ns#type https://bag.nl/id/buildingfunction/03440100000599322010061000000000N000
Lange Hagelstraat 60 3531BK Utrecht http://bag.nl/def/bag#address https://bag.nl/id/address/034

Let's get the address and building function related to one specific building.

In [139]:
buildingIRI = "<https://bag.nl/id/building/0344100000065003>"
response = g.query("SELECT * WHERE { %s ?p ?o .}" % buildingIRI)

for row in response:
    print(buildingIRI + "%s %s" % row)

<https://bag.nl/id/building/0344100000065003>https://bag.nl/id/address/0344010000059930 http://bag.nl/def/bag#hasAddress
<https://bag.nl/id/building/0344100000065003>https://bag.nl/id/buildingfunction/03440100001490952010061000000000N000 http://bag.nl/def/bag#hasFunction
<https://bag.nl/id/building/0344100000065003>POLYGON ((5.106746214649831 52.09240553536949, 5.106755223736506 52.09238599876256, 5.106767986939192 52.09235652129271, 5.106718096501726 52.0923472078326, 5.106717327523013 52.09234883284361, 5.106572197217012 52.09232278164907, 5.106549879822784 52.09237026642134, 5.106746214649831 52.09240553536949)) http://www.opengis.net/ont/geosparql#asWKT
<https://bag.nl/id/building/0344100000065003>http://bag.nl/def/bag#Building http://www.w3.org/1999/02/22-rdf-syntax-ns#type
<https://bag.nl/id/building/0344100000065003>Pand in gebruik http://bag.nl/def/bag#hasStatus
<https://bag.nl/id/building/0344100000065003>https://bag.nl/id/address/0344010000149095 http://bag.nl/def/bag#hasAddr

The building is of the type "Building". Furthermore, we can see that the building has two addresses, two building functions and a status. We only get information attached to the building. Perhaps we want to get information on the building function.

In [140]:
buildingIRI = "<https://bag.nl/id/building/0344100000065003>"
response = g.query(("SELECT ?BuildingFunction WHERE { %s <http://bag.nl/def/bag#hasFunction> ?BuildingFunction .}") % buildingIRI)

for row in response:
    print("%s" % row)

In [141]:
buildingIRI = "<https://bag.nl/id/building/0344100000065003>"
response = g.query(("SELECT ?BuildingFunction ?function WHERE { " +
                    "%s <http://bag.nl/def/bag#hasFunction> ?BuildingFunction ." +
                    "?BuildingFunction bag:function ?function "
                    "}") % buildingIRI)

for row in response:
    print(buildingIRI + " has %s, which has the function %s" % row)

<https://bag.nl/id/building/0344100000065003> has https://bag.nl/id/buildingfunction/03440100001490952010061000000000N000, which has the function woonfunctie
<https://bag.nl/id/building/0344100000065003> has https://bag.nl/id/buildingfunction/03440100000599302010061000000000N000, which has the function woonfunctie


Okay now we know this specific building has the "woonfunctie", which is residential function. 
Without linking the data we could not have know that this building was used for residential purposes.

In [142]:
buildingIRI = "<https://bag.nl/id/building/0344100000065003>"
response = g.query(("SELECT ?BuildingFunction ?function WHERE { " +
                    "%s <http://bag.nl/def/bag#hasFunction> ?BuildingFunction ." +
                    "?BuildingFunction bag:function ?function "
                    "}") % buildingIRI)

for row in response:
    print(buildingIRI + " has %s, which has the function %s" % row)

Let's invert the relation and say that we only want to plot the buildings with residential function on the map. By specifying SELECT DISTINCT we only get unique buildings back.

In [143]:
response = g.query(('''SELECT DISTINCT ?building WHERE { ''' +
                    '''?building <http://bag.nl/def/bag#hasFunction> ?BuildingFunction .''' +
                    '''?BuildingFunction bag:function "woonfunctie" .}'''))

for row in response:
    print("%s has residential function" % row)

https://bag.nl/id/building/0344100000005566 has residential function
https://bag.nl/id/building/0344100000065003 has residential function
https://bag.nl/id/building/0344100000036781 has residential function
https://bag.nl/id/building/0344100000007267 has residential function
https://bag.nl/id/building/0344100000073254 has residential function
https://bag.nl/id/building/0344100000026216 has residential function
https://bag.nl/id/building/0344100000063761 has residential function
https://bag.nl/id/building/0344100000073812 has residential function
https://bag.nl/id/building/0344100000068265 has residential function
https://bag.nl/id/building/0344100000036193 has residential function
https://bag.nl/id/building/0344100000016063 has residential function
https://bag.nl/id/building/0344100000072349 has residential function
https://bag.nl/id/building/0344100000066786 has residential function
https://bag.nl/id/building/0344100000011024 has residential function
https://bag.nl/id/building/0344100

Now we want to visualize these buildings with a residential function. The building geometries are extracted with SPARQL.

In [144]:
responses = g.query(('''SELECT DISTINCT ?buildingGeom WHERE { ''' +
                    '''?building <http://bag.nl/def/bag#hasFunction> ?BuildingFunction .''' +
                    '''?BuildingFunction bag:function "woonfunctie" .''' +
                    '''?building geosparql:asWKT ?buildingGeom}'''))

buildingGeomList = []
for response in responses:
    text = "%s" % response
    buildingGeomList.append(str(text))
    
buildingsResidential = pd.Series(buildingGeomList)
geometry = buildingsResidential.apply(shapely.wkt.loads)

buildingsResidential.apply(shapely.wkt.loads)
crs = {'init': 'epsg:4326'}
buildingsResidential_gdf = gpd.GeoDataFrame(buildingsResidential, crs=crs, geometry=geometry)


In [145]:
m = folium.Map([52.0928, 5.1062], zoom_start=18, tiles='cartodbpositron')
folium.GeoJson(buildings.geometry, name="Buildings", style_function=lambda feature: {'fillColor': 'White', 'color': 'Red'}).add_to(m)
folium.GeoJson(buildingsResidential_gdf.geometry, name="Buildings with residential function", style_function=lambda feature: {'fillColor': 'White', 'color': 'Green'}).add_to(m)
folium.LatLngPopup().add_to(m)
folium.LayerControl().add_to(m)
m

<div style="width:100%;"><div style="position:relative;width:100%;height:0;padding-bottom:60%;"><iframe src="data:text/html;charset=utf-8;base64,PCFET0NUWVBFIGh0bWw+CjxoZWFkPiAgICAKICAgIDxtZXRhIGh0dHAtZXF1aXY9ImNvbnRlbnQtdHlwZSIgY29udGVudD0idGV4dC9odG1sOyBjaGFyc2V0PVVURi04IiAvPgogICAgPHNjcmlwdD5MX1BSRUZFUl9DQU5WQVM9ZmFsc2U7IExfTk9fVE9VQ0g9ZmFsc2U7IExfRElTQUJMRV8zRD1mYWxzZTs8L3NjcmlwdD4KICAgIDxzY3JpcHQgc3JjPSJodHRwczovL2Nkbi5qc2RlbGl2ci5uZXQvbnBtL2xlYWZsZXRAMS4yLjAvZGlzdC9sZWFmbGV0LmpzIj48L3NjcmlwdD4KICAgIDxzY3JpcHQgc3JjPSJodHRwczovL2FqYXguZ29vZ2xlYXBpcy5jb20vYWpheC9saWJzL2pxdWVyeS8xLjExLjEvanF1ZXJ5Lm1pbi5qcyI+PC9zY3JpcHQ+CiAgICA8c2NyaXB0IHNyYz0iaHR0cHM6Ly9tYXhjZG4uYm9vdHN0cmFwY2RuLmNvbS9ib290c3RyYXAvMy4yLjAvanMvYm9vdHN0cmFwLm1pbi5qcyI+PC9zY3JpcHQ+CiAgICA8c2NyaXB0IHNyYz0iaHR0cHM6Ly9jZG5qcy5jbG91ZGZsYXJlLmNvbS9hamF4L2xpYnMvTGVhZmxldC5hd2Vzb21lLW1hcmtlcnMvMi4wLjIvbGVhZmxldC5hd2Vzb21lLW1hcmtlcnMuanMiPjwvc2NyaXB0PgogICAgPGxpbmsgcmVsPSJzdHlsZXNoZWV0IiBocmVmPSJodHRwczovL2Nkbi5qc2RlbGl2ci5uZXQvbnBtL2xlYWZsZXRAMS4yLjAvZGlzdC9sZWFmbGV0LmNzcyIvPgogICAgPGxpbmsgcmVsPSJzdHlsZXNoZWV0IiBocmVmPSJodHRwczovL21heGNkbi5ib290c3RyYXBjZG4uY29tL2Jvb3RzdHJhcC8zLjIuMC9jc3MvYm9vdHN0cmFwLm1pbi5jc3MiLz4KICAgIDxsaW5rIHJlbD0ic3R5bGVzaGVldCIgaHJlZj0iaHR0cHM6Ly9tYXhjZG4uYm9vdHN0cmFwY2RuLmNvbS9ib290c3RyYXAvMy4yLjAvY3NzL2Jvb3RzdHJhcC10aGVtZS5taW4uY3NzIi8+CiAgICA8bGluayByZWw9InN0eWxlc2hlZXQiIGhyZWY9Imh0dHBzOi8vbWF4Y2RuLmJvb3RzdHJhcGNkbi5jb20vZm9udC1hd2Vzb21lLzQuNi4zL2Nzcy9mb250LWF3ZXNvbWUubWluLmNzcyIvPgogICAgPGxpbmsgcmVsPSJzdHlsZXNoZWV0IiBocmVmPSJodHRwczovL2NkbmpzLmNsb3VkZmxhcmUuY29tL2FqYXgvbGlicy9MZWFmbGV0LmF3ZXNvbWUtbWFya2Vycy8yLjAuMi9sZWFmbGV0LmF3ZXNvbWUtbWFya2Vycy5jc3MiLz4KICAgIDxsaW5rIHJlbD0ic3R5bGVzaGVldCIgaHJlZj0iaHR0cHM6Ly9yYXdnaXQuY29tL3B5dGhvbi12aXN1YWxpemF0aW9uL2ZvbGl1bS9tYXN0ZXIvZm9saXVtL3RlbXBsYXRlcy9sZWFmbGV0LmF3ZXNvbWUucm90YXRlLmNzcyIvPgogICAgPHN0eWxlPmh0bWwsIGJvZHkge3dpZHRoOiAxMDAlO2hlaWdodDogMTAwJTttYXJnaW46IDA7cGFkZGluZzogMDt9PC9zdHlsZT4KICAgIDxzdHlsZT4jbWFwIHtwb3NpdGlvbjphYnNvbHV0ZTt0b3A6MDtib3R0b206MDtyaWdodDowO2xlZnQ6MDt9PC9zdHlsZT4KICAgIAogICAgPHN0eWxlPiNtYXBfYTFlNjQyYTkwNWY2NDUxZThhNGU4NjA0NDVkNWZhOWQgewogICAgICAgIHBvc2l0aW9uOiByZWxhdGl2ZTsKICAgICAgICB3aWR0aDogMTAwLjAlOwogICAgICAgIGhlaWdodDogMTAwLjAlOwogICAgICAgIGxlZnQ6IDAuMCU7CiAgICAgICAgdG9wOiAwLjAlOwogICAgICAgIH0KICAgIDwvc3R5bGU+CjwvaGVhZD4KPGJvZHk+ICAgIAogICAgCiAgICA8ZGl2IGNsYXNzPSJmb2xpdW0tbWFwIiBpZD0ibWFwX2ExZTY0MmE5MDVmNjQ1MWU4YTRlODYwNDQ1ZDVmYTlkIiA+PC9kaXY+CjwvYm9keT4KPHNjcmlwdD4gICAgCiAgICAKICAgIAogICAgICAgIHZhciBib3VuZHMgPSBudWxsOwogICAgCgogICAgdmFyIG1hcF9hMWU2NDJhOTA1ZjY0NTFlOGE0ZTg2MDQ0NWQ1ZmE5ZCA9IEwubWFwKAogICAgICAgICdtYXBfYTFlNjQyYTkwNWY2NDUxZThhNGU4NjA0NDVkNWZhOWQnLCB7CiAgICAgICAgY2VudGVyOiBbNTIuMDkyOCwgNS4xMDYyXSwKICAgICAgICB6b29tOiAxOCwKICAgICAgICBtYXhCb3VuZHM6IGJvdW5kcywKICAgICAgICBsYXllcnM6IFtdLAogICAgICAgIHdvcmxkQ29weUp1bXA6IGZhbHNlLAogICAgICAgIGNyczogTC5DUlMuRVBTRzM4NTcsCiAgICAgICAgem9vbUNvbnRyb2w6IHRydWUsCiAgICAgICAgfSk7CgogICAgCiAgICAKICAgIHZhciB0aWxlX2xheWVyX2E3YzYwODBkYmQ5NzQyYTNiOWNiMDQ1NjA0ZTk4YjhiID0gTC50aWxlTGF5ZXIoCiAgICAgICAgJ2h0dHBzOi8vY2FydG9kYi1iYXNlbWFwcy17c30uZ2xvYmFsLnNzbC5mYXN0bHkubmV0L2xpZ2h0X2FsbC97en0ve3h9L3t5fS5wbmcnLAogICAgICAgIHsKICAgICAgICAiYXR0cmlidXRpb24iOiBudWxsLAogICAgICAgICJkZXRlY3RSZXRpbmEiOiBmYWxzZSwKICAgICAgICAibWF4TmF0aXZlWm9vbSI6IDE4LAogICAgICAgICJtYXhab29tIjogMTgsCiAgICAgICAgIm1pblpvb20iOiAwLAogICAgICAgICJub1dyYXAiOiBmYWxzZSwKICAgICAgICAic3ViZG9tYWlucyI6ICJhYmMiCn0pLmFkZFRvKG1hcF9hMWU2NDJhOTA1ZjY0NTFlOGE0ZTg2MDQ0NWQ1ZmE5ZCk7CiAgICAKICAgICAgICAKICAgICAgICB2YXIgZ2VvX2pzb25fOGRkZjk4ZWI5ZmEzNDkyYThlY2EzY2JjZGIxODg0NWIgPSBMLmdlb0pzb24oCiAgICAgICAgICAgIHsiYmJveCI6IFs1LjEwNTYwODUxNDIzMzQ5NywgNTIuMDkyMTAwMTAzNjU4ODgsIDUuMTA2ODYyNzg0MDY4NjExLCA1Mi4wOTMwMzUxNjQxNjcyNzZdLCAiZmVhdHVyZXMiOiBbeyJiYm94IjogWzUuMTA2NTI5NTczODkzODMzLCA1Mi4wOTIzNzAyNjY0MjEzNCwgNS4xMDY3NDYyMTQ2NDk4MzEsIDUyLjA5MjQ0MjIyODU4MTRdLCAiZ2VvbWV0cnkiOiB7ImNvb3JkaW5hdGVzIjogW1tbNS4xMDY1NDk4Nzk4MjI3ODQsIDUyLjA5MjM3MDI2NjQyMTM0XSwgWzUuMTA2NTI5NTczODkzODMzLCA1Mi4wOTI0MTM0NTA2OTI2OTZ

Ok cool we have the buildings with a residential function, but we also want to know the address of these buildings. So we can add the geometry and address of the address.

In [146]:
responses = g.query(('''SELECT DISTINCT ?addressGeom ?addressText WHERE { ''' +
                    '''?building <http://bag.nl/def/bag#hasFunction> ?BuildingFunction .''' +
                    '''?BuildingFunction bag:function "woonfunctie" .''' +
                    '''?building <http://bag.nl/def/bag#hasAddress> ?Address .''' +
                    '''?Address geosparql:asWKT ?addressGeom .''' +
                    '''?Address bag:address ?addressText}'''))

addressGeomList = []
addressTextList = []
for response in responses:
    addressGeom = "%s" % str(response.addressGeom.toPython())
    addressText = "%s" % str(response.addressText.toPython())
    addressGeomList.append(addressGeom)
    addressTextList.append(addressText)

addresses = pd.DataFrame({"addressGeom": addressGeomList, "addressText": addressTextList})
geometry = addresses['addressGeom'].apply(shapely.wkt.loads)
addresses.drop(columns=['addressGeom'], inplace=True)
crs = {'init': 'epsg:4326'}
addresses_gdf = gpd.GeoDataFrame(addresses, crs=crs, geometry=geometry)

In [147]:
m = folium.Map([52.0928, 5.1062], zoom_start=18, tiles='cartodbpositron')
icon = folium.Icon(color='black')
addressesGroup = FeatureGroup(name="Addresses")

for index, address in addresses_gdf.iterrows():
    lon, lat = address.geometry.coords.xy
    Marker([lat[0], lon[0]], icon=folium.Icon(color="black", icon='info-sign'), popup=address['addressText']).add_to(m)
#     addressesGroup.add_child(marker)
# m.add_child(addressesGroup)
folium.GeoJson(buildings.geometry, name="Buildings", style_function=lambda feature: {'fillColor': 'White', 'color': 'Red'}).add_to(m)
folium.GeoJson(buildingsResidential_gdf.geometry, name="Buildings with residential function", style_function=lambda feature: {'fillColor': 'White', 'color': 'Green'}).add_to(m)
folium.LayerControl().add_to(m)
m

<div style="width:100%;"><div style="position:relative;width:100%;height:0;padding-bottom:60%;"><iframe src="data:text/html;charset=utf-8;base64,PCFET0NUWVBFIGh0bWw+CjxoZWFkPiAgICAKICAgIDxtZXRhIGh0dHAtZXF1aXY9ImNvbnRlbnQtdHlwZSIgY29udGVudD0idGV4dC9odG1sOyBjaGFyc2V0PVVURi04IiAvPgogICAgPHNjcmlwdD5MX1BSRUZFUl9DQU5WQVM9ZmFsc2U7IExfTk9fVE9VQ0g9ZmFsc2U7IExfRElTQUJMRV8zRD1mYWxzZTs8L3NjcmlwdD4KICAgIDxzY3JpcHQgc3JjPSJodHRwczovL2Nkbi5qc2RlbGl2ci5uZXQvbnBtL2xlYWZsZXRAMS4yLjAvZGlzdC9sZWFmbGV0LmpzIj48L3NjcmlwdD4KICAgIDxzY3JpcHQgc3JjPSJodHRwczovL2FqYXguZ29vZ2xlYXBpcy5jb20vYWpheC9saWJzL2pxdWVyeS8xLjExLjEvanF1ZXJ5Lm1pbi5qcyI+PC9zY3JpcHQ+CiAgICA8c2NyaXB0IHNyYz0iaHR0cHM6Ly9tYXhjZG4uYm9vdHN0cmFwY2RuLmNvbS9ib290c3RyYXAvMy4yLjAvanMvYm9vdHN0cmFwLm1pbi5qcyI+PC9zY3JpcHQ+CiAgICA8c2NyaXB0IHNyYz0iaHR0cHM6Ly9jZG5qcy5jbG91ZGZsYXJlLmNvbS9hamF4L2xpYnMvTGVhZmxldC5hd2Vzb21lLW1hcmtlcnMvMi4wLjIvbGVhZmxldC5hd2Vzb21lLW1hcmtlcnMuanMiPjwvc2NyaXB0PgogICAgPGxpbmsgcmVsPSJzdHlsZXNoZWV0IiBocmVmPSJodHRwczovL2Nkbi5qc2RlbGl2ci5uZXQvbnBtL2xlYWZsZXRAMS4yLjAvZGlzdC9sZWFmbGV0LmNzcyIvPgogICAgPGxpbmsgcmVsPSJzdHlsZXNoZWV0IiBocmVmPSJodHRwczovL21heGNkbi5ib290c3RyYXBjZG4uY29tL2Jvb3RzdHJhcC8zLjIuMC9jc3MvYm9vdHN0cmFwLm1pbi5jc3MiLz4KICAgIDxsaW5rIHJlbD0ic3R5bGVzaGVldCIgaHJlZj0iaHR0cHM6Ly9tYXhjZG4uYm9vdHN0cmFwY2RuLmNvbS9ib290c3RyYXAvMy4yLjAvY3NzL2Jvb3RzdHJhcC10aGVtZS5taW4uY3NzIi8+CiAgICA8bGluayByZWw9InN0eWxlc2hlZXQiIGhyZWY9Imh0dHBzOi8vbWF4Y2RuLmJvb3RzdHJhcGNkbi5jb20vZm9udC1hd2Vzb21lLzQuNi4zL2Nzcy9mb250LWF3ZXNvbWUubWluLmNzcyIvPgogICAgPGxpbmsgcmVsPSJzdHlsZXNoZWV0IiBocmVmPSJodHRwczovL2NkbmpzLmNsb3VkZmxhcmUuY29tL2FqYXgvbGlicy9MZWFmbGV0LmF3ZXNvbWUtbWFya2Vycy8yLjAuMi9sZWFmbGV0LmF3ZXNvbWUtbWFya2Vycy5jc3MiLz4KICAgIDxsaW5rIHJlbD0ic3R5bGVzaGVldCIgaHJlZj0iaHR0cHM6Ly9yYXdnaXQuY29tL3B5dGhvbi12aXN1YWxpemF0aW9uL2ZvbGl1bS9tYXN0ZXIvZm9saXVtL3RlbXBsYXRlcy9sZWFmbGV0LmF3ZXNvbWUucm90YXRlLmNzcyIvPgogICAgPHN0eWxlPmh0bWwsIGJvZHkge3dpZHRoOiAxMDAlO2hlaWdodDogMTAwJTttYXJnaW46IDA7cGFkZGluZzogMDt9PC9zdHlsZT4KICAgIDxzdHlsZT4jbWFwIHtwb3NpdGlvbjphYnNvbHV0ZTt0b3A6MDtib3R0b206MDtyaWdodDowO2xlZnQ6MDt9PC9zdHlsZT4KICAgIAogICAgPHN0eWxlPiNtYXBfYmZkYTlkNjhhZjQzNDNjNWE0ZTg5YTYwYzU3YWE3NzcgewogICAgICAgIHBvc2l0aW9uOiByZWxhdGl2ZTsKICAgICAgICB3aWR0aDogMTAwLjAlOwogICAgICAgIGhlaWdodDogMTAwLjAlOwogICAgICAgIGxlZnQ6IDAuMCU7CiAgICAgICAgdG9wOiAwLjAlOwogICAgICAgIH0KICAgIDwvc3R5bGU+CjwvaGVhZD4KPGJvZHk+ICAgIAogICAgCiAgICA8ZGl2IGNsYXNzPSJmb2xpdW0tbWFwIiBpZD0ibWFwX2JmZGE5ZDY4YWY0MzQzYzVhNGU4OWE2MGM1N2FhNzc3IiA+PC9kaXY+CjwvYm9keT4KPHNjcmlwdD4gICAgCiAgICAKICAgIAogICAgICAgIHZhciBib3VuZHMgPSBudWxsOwogICAgCgogICAgdmFyIG1hcF9iZmRhOWQ2OGFmNDM0M2M1YTRlODlhNjBjNTdhYTc3NyA9IEwubWFwKAogICAgICAgICdtYXBfYmZkYTlkNjhhZjQzNDNjNWE0ZTg5YTYwYzU3YWE3NzcnLCB7CiAgICAgICAgY2VudGVyOiBbNTIuMDkyOCwgNS4xMDYyXSwKICAgICAgICB6b29tOiAxOCwKICAgICAgICBtYXhCb3VuZHM6IGJvdW5kcywKICAgICAgICBsYXllcnM6IFtdLAogICAgICAgIHdvcmxkQ29weUp1bXA6IGZhbHNlLAogICAgICAgIGNyczogTC5DUlMuRVBTRzM4NTcsCiAgICAgICAgem9vbUNvbnRyb2w6IHRydWUsCiAgICAgICAgfSk7CgogICAgCiAgICAKICAgIHZhciB0aWxlX2xheWVyXzRmZmJkNTBjZDM2YTRhZmE5MDliNjBkODg3MmJiYzM4ID0gTC50aWxlTGF5ZXIoCiAgICAgICAgJ2h0dHBzOi8vY2FydG9kYi1iYXNlbWFwcy17c30uZ2xvYmFsLnNzbC5mYXN0bHkubmV0L2xpZ2h0X2FsbC97en0ve3h9L3t5fS5wbmcnLAogICAgICAgIHsKICAgICAgICAiYXR0cmlidXRpb24iOiBudWxsLAogICAgICAgICJkZXRlY3RSZXRpbmEiOiBmYWxzZSwKICAgICAgICAibWF4TmF0aXZlWm9vbSI6IDE4LAogICAgICAgICJtYXhab29tIjogMTgsCiAgICAgICAgIm1pblpvb20iOiAwLAogICAgICAgICJub1dyYXAiOiBmYWxzZSwKICAgICAgICAic3ViZG9tYWlucyI6ICJhYmMiCn0pLmFkZFRvKG1hcF9iZmRhOWQ2OGFmNDM0M2M1YTRlODlhNjBjNTdhYTc3Nyk7CiAgICAKICAgICAgICB2YXIgbWFya2VyXzk4ZGRkZTkyNGM5MjQzZTg4YTIxMmVlODZlZGU2ZWFiID0gTC5tYXJrZXIoCiAgICAgICAgICAgIFs1Mi4wOTI0MDIzMTkwNDU2NiwgNS4xMDY1ODA1ODQ4ODIzNzNdLAogICAgICAgICAgICB7CiAgICAgICAgICAgICAgICBpY29uOiBuZXcgTC5JY29uLkRlZmF1bHQoKQogICAgICAgICAgICAgICAgfQogICAgICAgICAgICApLmFkZFRvKG1hcF9iZmRhOWQ2OGFmNDM0M2M1YTRlODlhNjBjNTdhYTc3Nyk7CiAgICAgICAgCiAgICAKCiAgICAgICAgICAgICAgICB2YXIgaWNvbl9kMjVmNGE1ZWU2YWY0ZTM2YTYwM2RjYmUwMWNhZjc4YyA9IEwuQXdlc29tZU1hcmtlcnMuaWNvbih7CiAgICAgICAgICAgICAgICAgICAgaWNvbjogJ2luZm8tc2l

Hopefully the tutorial was interesting. Another [recommended SPARQL tutorial](https://wouterbeek.github.io/presentation/SPARQL-Tutorial.html#/) to continue learning is from Wouter Beek at Triply/VU Amsterdam that developed 3D visualizations for GeoSPARQL queries. ![3D GeoSPARLQ](https://wouterbeek.github.io/resource/img/3d/boelelaan.png)